In [75]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
import xgboost as xgb
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('../data/trainPrepDone.csv')
test = pd.read_csv('../data/testPrepDone.csv')
testResults = pd.read_csv('../data/trueResults.csv')

In [3]:
train = train.fillna(0)
test = test.fillna(0)

In [4]:
train['culmative_workers_success'] = train.apply(lambda row: row.director_success + row.producer_success + row.actor1_success + row.actor2_success, axis = 1)
test['culmative_workers_success'] = test.apply(lambda row: row.director_success + row.producer_success + row.actor1_success + row.actor2_success, axis = 1)

In [5]:
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]
test = test.merge(testResults, on='id')

In [13]:
test.drop('Unnamed: 0', axis=1, inplace=True)
train.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
test['log_revenue'] = np.log(test['revenue'])

/Users/hyunchoi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [21]:
test.drop('revenue', axis=1, inplace=True)
train.drop('revenue', axis=1, inplace=True)

In [37]:
data = test.append(train)

In [53]:
data.shape

(5950, 26)

In [45]:
data = data.loc[data.log_revenue != 0]
data = data.loc[data.log_revenue != -np.inf]

In [80]:
datax = data[['Action', 'Adventure', 'Animation', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Foreign', 'Mystery', 'Science Fiction', 'Thriller',
       'Western', 'actor1_success', 'actor2_success', 'adjusted_budget'
    , 'culmative_workers_success', 'director_success',
       'has_collection', 'id', 'log_budget', 'popularity',
       'producer_success', 'release_year', 'runtime']]

In [61]:
datay = data['log_revenue']

In [62]:
kf = KFold(n_splits=9)
kf.get_n_splits(datax)

9

In [105]:
lm_scores = []
rf_scores = []
xgb_scores = []
for train_index, test_index in kf.split(datax):
    #Training simple linear regression model
    lm = linear_model.LinearRegression()
    lin_model = lm.fit(datax.iloc[train_index], datay.iloc[train_index])
    lin_predictions = lm.predict(datax.iloc[test_index])
    lm_scores.append(r2_score(datay.iloc[test_index], lin_predictions))
    #Training Random Forest Regressor
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf_model = rf.fit(datax.iloc[train_index], datay.iloc[train_index])
    rf_predictions = rf.predict(datax.iloc[test_index])
    rf_scores.append(r2_score(datay.iloc[test_index], rf_predictions))
    #Training XGboost model
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 15, alpha = 15, n_estimators = 100)
    xg_reg.fit(datax.iloc[train_index], datay.iloc[train_index])
    xg_predictions = xg_reg.predict(datax.iloc[test_index])
    xgb_scores.append(r2_score(datay.iloc[test_index], xg_predictions))

/Users/hyunchoi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hyunchoi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hyunchoi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hyunchoi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hyunchoi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(dat

In [106]:
print(np.mean(lm_scores))
print(np.mean(rf_scores))
print(np.mean(xgb_scores))

0.3932606798722514
0.5232514617462455
0.5004075747699532
